<a href="https://colab.research.google.com/github/Anjana2002/Language-Identification-for-Malayalam-English-Code-Mixed-Text/blob/main/modeling/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BERT MODELING


In [5]:
!pip install transformers
!pip install simpletransformers


In [9]:
import pandas as pd
import ast
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [10]:
df = pd.read_csv('annotated_comments.csv')
df.head()

,video_id,text,annotated
0,63i4ZQcvpcQ,nayanthara kettiyathin shesham kanunnavar,"[('nayanthara', 'mal'), ('kettiyathin', 'mal')..."
1,63i4ZQcvpcQ,00:12 which is that song playing in the backgr...,"[('00:12', 'univ'), ('which', 'eng'), ('is', '..."
2,63i4ZQcvpcQ,2024 july 21n kaanunna njan🥲🥲,"[('2024', 'univ'), ('july', 'eng'), ('21n', 'u..."
3,63i4ZQcvpcQ,15:57 what she is saying,"[('15:57', 'univ'), ('what', 'eng'), ('she', '..."
4,63i4ZQcvpcQ,😊 ara e script oke ezhuthiye..nice dialogues a...,"[('😊', 'univ'), ('ara', 'mal'), ('e', 'undef')..."


In [11]:
#rename video_id to sentence_id
df = df.rename(columns={'video_id':'sentence_id'})
df['sentence_id'] = range(1, len(df) + 1)
#tokenize text
df['text'] = df['text'].apply(word_tokenize)
df.head()

,sentence_id,text,annotated
0,1,"[nayanthara, kettiyathin, shesham, kanunnavar]","[('nayanthara', 'mal'), ('kettiyathin', 'mal')..."
1,2,"[00:12, which, is, that, song, playing, in, th...","[('00:12', 'univ'), ('which', 'eng'), ('is', '..."
2,3,"[2024, july, 21n, kaanunna, njan🥲🥲]","[('2024', 'univ'), ('july', 'eng'), ('21n', 'u..."
3,4,"[15:57, what, she, is, saying]","[('15:57', 'univ'), ('what', 'eng'), ('she', '..."
4,5,"[😊, ara, e, script, oke, ezhuthiye, .., nice, ...","[('😊', 'univ'), ('ara', 'mal'), ('e', 'undef')..."


In [12]:
formatted_data = []

for _, row in df.iterrows():
    sentence_id = row["sentence_id"]

    # Convert the `annotated` string to a list of tuples
    annotations = ast.literal_eval(row["annotated"]) if isinstance(row["annotated"], str) else row["annotated"]

    # Iterate over the annotations and extract words and labels
    for item in annotations:
        if isinstance(item, tuple) and len(item) == 2:  # Ensure it is a valid (word, label) tuple
            token, label = item
            if label != "undef":  # Exclude tokens with the label "undef"
                formatted_data.append({"sentence_id": sentence_id, "words": token, "labels": label})
            # formatted_data.append({"sentence_id": sentence_id, "words": token, "labels": label})

# Convert the formatted data to a DataFrame
df = pd.DataFrame(formatted_data)

# Save the formatted dataset to a new CSV file
df.to_csv("ner_dataset.csv", index=False)
print("Formatted dataset saved as 'ner_dataset.csv'")

Formatted dataset saved as 'ner_dataset.csv'


In [13]:
df.head()

,sentence_id,words,labels
0,1,nayanthara,mal
1,1,kettiyathin,mal
2,1,shesham,mal
3,1,kanunnavar,mal
4,2,00:12,univ


In [14]:

# Split into training (80%), validation (10%), and test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

print(f"Training size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")


Training size: 263461
Validation size: 32933
Test size: 32933


In [15]:
from transformers import XLMRobertaTokenizer
import torch
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [16]:
# Extract unique labels from the dataset
unique_labels = list(df["labels"].unique())
print("Unique labels in the dataset:", unique_labels)


Unique labels in the dataset: ['mal', 'univ', 'eng', 'mix', 'acr']


In [17]:

from simpletransformers.ner import NERModel
import logging

# Enable logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define model configuration
model_args = {
    "overwrite_output_dir": True,
    "reprocess_input_data": True,
    "save_steps": -1,
    "num_train_epochs": 5,
    "train_batch_size": 16,
    "eval_batch_size": 16,
    "max_seq_length": 128,
    "learning_rate": 2e-5,
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "evaluate_during_training": True,
    "output_dir": "outputs/",
    "best_model_dir": "outputs/best_model/",
    "labels_list": unique_labels,  # Add this line with your labels
}
model_1 = NERModel(
    model_type="bert",
    model_name="bert-base-uncased",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

model_2 = NERModel(
    model_type="xlmroberta",
    model_name="xlm-roberta-base",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)
model_3 = NERModel(
    model_type="camembert",
    model_name="camembert-base",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)
model_4 = NERModel(
    model_type="distilbert",
    model_name="distilbert-base-uncased",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)
model_5 = NERModel(
    model_type="electra",
    model_name="google/electra-small-discriminator",
    args=model_args,
    use_cuda=torch.cuda.is_available()
)



print("Transformer model for NER initialized successfully!")


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Transformer model for NER initialized successfully!


In [18]:
train_data = train_df[["sentence_id", "words", "labels"]]
val_data = val_df[["sentence_id", "words", "labels"]]
print(train_data.head())
print(val_data.head())

        sentence_id   words labels
199781        33943  aanenn    mal
131447        21490       %   univ
204812        34786     dhe    mal
27121          4977    next    eng
85254         14281   robin    eng
        sentence_id      words labels
258702        44189        njn    mal
56139          9908      react    eng
198375        33702          🤣   univ
324160        53398  thonnunn🤗    mal
263637        45105      😍😍😂😂😀   univ


In [19]:
# Ensure labels are consistent with the defined `labels_list`
assert set(train_data["labels"].unique()).issubset(set(unique_labels)), "Mismatch in train labels"
assert set(val_data["labels"].unique()).issubset(set(unique_labels)), "Mismatch in val labels"
model_1.train_model(train_data, eval_data=val_data)



  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/3305 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model_2.train_model(train_data, eval_data=val_data)

In [ ]:
model_3.train_model(train_data, eval_data=val_data)

In [ ]:
model_4.train_model(train_data, eval_data=val_data)

In [ ]:
model_5.train_model(train_data, eval_data=val_data)

In [ ]:

example_sentence = "njn enn avide poyi, videoyil njn kandu, comedyu"
predictions, raw_outputs = model_1.predict([example_sentence])  # Pass it as a list of strings
print("Predictions:", predictions)
predictions, raw_outputs = model_2.predict([example_sentence])
print("Predictions:", predictions)
predictions, raw_outputs = model_3.predict([example_sentence])
print("Predictions:", predictions)
predictions, raw_outputs = model_4.predict([example_sentence])
print("Predictions:", predictions)
predictions, raw_outputs = model_5.predict([example_sentence])
print("Predictions:", predictions)